# Libraries

In [26]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.util import ngrams
import pymorphy2

In [5]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

# Functions

In [7]:
def return_bigrams(text):
    double_word = []
    for grams in ngrams(text.split(), 2):
        double_word.append(grams)
    return double_word
   # double_word = double_word[:-1]
    
def return_trigrams(text):
    triple_word = []
    for grams in ngrams(text.split(), 3):
        triple_word.append(grams)
    return triple_word
   # double_word = double_word[:-1]

def return_fograms(text):
    quadro_word = []
    for grams in ngrams(text.split(), 4):
        quadro_word.append(grams)
    return quadro_word

In [8]:
def counter(df1):
    all_bigrams = []
    for i in df1.index: # проходимся и добавляем по каждому слову в листе в каждой строке
        for j in df1['bigram_list'][i]:
            all_bigrams.append(j)
    
    all_trigrams = []
    for i in df1.index:
        for j in df1['trigram_list'][i]:
            all_trigrams.append(j)

    all_fograms = []
    for i in df1.index:
        for j in df1['fogram_list'][i]:
            all_fograms.append(j)

    temp_df_bi = pd.DataFrame(all_bigrams).value_counts().head(60).to_frame('count').reset_index()
    temp_df_tri = pd.DataFrame(all_trigrams).value_counts().head(30).to_frame('count').reset_index()
    temp_df_four = pd.DataFrame(all_fograms).value_counts().head(30).to_frame('count').reset_index()

    return temp_df_bi, temp_df_tri, temp_df_four

In [9]:
def occurences(text, words):
    in_dict = any([word in text for word in words])
    if in_dict == True:
        return True
    else:
        return False

In [10]:
def listToString(s):
    str1 = ""
    for ele in s:
        str1 += ele + ' '
    return str1

In [11]:
greetings = {'здравств', 'добрый'}
introduction = {'звать'}
manager_name = {}
company_name = {}
goodbye = {'хороший вечер', 'до свидание', 'весь добрый'}


# Proccessing

In [12]:
df = pd.read_csv('test_data.csv')

df['is_greeting'] = ''
df['is_introduction'] = ''
df['manager_name'] = ''
df['company_name'] = ''
df['is_goodbye'] = ''

df['lemmatized'] = ''


#tokenizing and lemmatizing text
for i in df.index:
    doc2 = Doc(df['text'][i])
    doc2.segment(segmenter)
    doc2.tag_morph(morph_tagger)
    lemmatized_text = []
    for token in doc2.tokens:
        token.lemmatize(morph_vocab)
        lemmatized_text.append(token.lemma)
    df['lemmatized'][i] = lemmatized_text
        


df['lemmatized_string'] = df.apply(lambda x: listToString(x['lemmatized']), axis = 1)
df = df[df['lemmatized_string'].isnull() == False]
df['bigram_list'] = df.apply(lambda x: return_bigrams(x['lemmatized_string']), axis = 1)
df['trigram_list'] = df.apply(lambda x: return_trigrams(x['lemmatized_string']), axis = 1)
df['fogram_list'] = df.apply(lambda x: return_fograms(x['lemmatized_string']), axis = 1)

df['is_greeting'] = df['lemmatized_string'].apply(lambda x: True if occurences(x, greetings) else False)
df['is_introduction'] = df['lemmatized_string'].apply(lambda x: True if occurences(x, introduction) else False)
df['is_goodbye'] = df['lemmatized_string'].apply(lambda x: True if occurences(x, goodbye) else False)

# df1 = df.dropna(subset = ['text'])
# df1 = df1.astype({'value': 'int64'})

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [43]:
df['bigram_list'][1]

[('алло', 'здравствовать')]

In [22]:
df[df['lemmatized_string'].str.contains('звать', regex = False) == True]

,dlg_id,line_n,role,text,lemmatized,lemmatized_string,bigram_list,trigram_list,fogram_list
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне...",я звать ангелина компания диджитал бизнес звон...,"[(я, звать), (звать, ангелина), (ангелина, ком...","[(я, звать, ангелина), (звать, ангелина, компа...","[(я, звать, ангелина, компания), (звать, ангел..."
111,1,2,client,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне...",я звать ангелина компания диджитал бизнес звон...,"[(я, звать), (звать, ангелина), (ангелина, ком...","[(я, звать, ангелина), (звать, ангелина, компа...","[(я, звать, ангелина, компания), (звать, ангел..."
145,1,36,client,А могли бы назвать ваш номер аккаунта,"[а, мочь, бы, назвать, ваш, номер, аккаунт]",а мочь бы назвать ваш номер аккаунт,"[(а, мочь), (мочь, бы), (бы, назвать), (назват...","[(а, мочь, бы), (мочь, бы, назвать), (бы, назв...","[(а, мочь, бы, назвать), (мочь, бы, назвать, в..."
167,2,3,client,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне...",я звать ангелина компания диджитал бизнес звон...,"[(я, звать), (звать, ангелина), (ангелина, ком...","[(я, звать, ангелина), (звать, ангелина, компа...","[(я, звать, ангелина, компания), (звать, ангел..."
251,3,2,client,Добрый меня максим зовут компания китобизнес у...,"[добрый, я, максим, звать, компания, китобизне...",добрый я максим звать компания китобизнес удоб...,"[(добрый, я), (я, максим), (максим, звать), (з...","[(добрый, я, максим), (я, максим, звать), (мак...","[(добрый, я, максим, звать), (я, максим, звать..."


In [53]:
text = df['text'][111].title()
text

'Меня Зовут Ангелина Компания Диджитал Бизнес Звоню Вам По Поводу Продления А Мы Сели Обратила Внимание Что У Вас Срок Заканчивается'

In [63]:
#doc2 = Doc(df['text'][111])
doc2 = Doc(text)
# print(doc2)
doc2.segment(segmenter)
# print(doc2.tokens)
doc2.tag_morph(morph_tagger)
doc2.tag_ner(ner_tagger)
# print(doc2.spans[:5])
for span in doc2.spans:
        if span.type == PER:
            print(span.text)
            print(span)
# print(doc2.spans)
# df.iloc[111].lemmatized

Зовут Ангелина
DocSpan(start=5, stop=19, type='PER', text='Зовут Ангелина', tokens=[...])


In [64]:
for i in df.index:
    doc2 = Doc(df['text'][i].title())
    doc2.segment(segmenter)
    doc2.tag_morph(morph_tagger)
    doc2.tag_ner(ner_tagger)
    for span in doc2.spans:
        if span.type == PER:
            print(span.text)
#             span.extract_fact(names_extractor)

Зовут Ангелина
Обзванивать
Зовут Ангелина
Убрали
Угу Да Сейчас
Дэ Точка Ру Правильно
Денис Дэ Е Эн
Зовут Ангелина
Да Ага Вижу
Да Да Да
Например Кэшбэком
Угу Понял Хорошо
Скинуть
Да Да Да На
Сапаров Собачка
Нет Тоже Нету Да
Угу Хорошо
Угу Хорошо
Да Удобно
Анастасия Подсказала
Угу Все Хорошо
Да Так
Дам Ну Да Да Обсудите
Угу Да До Свидания
Анастасия
Да Да Да Да Да
Да Да Да Да Да
Да Да Да Здравствуйте
Анастасия Меня Слышно Да Хорошо
Да Слышно
Да Да Да
Да Да
Да Да Все Верно
Да Все Верно
Да Да
Ну Да Да
Да Да
Дмитрий
Да Хорошо
Уже Обсудите Конкретно Продам


In [27]:
prob_thresh = 0.4
morph = pymorphy2.MorphAnalyzer()
text = df.loc[3].text

for word in word_tokenize(text):
    for p in morph.parse(word):
        if 'Name' in p.tag and p.score >= prob_thresh:
            print('{:<12}\t({:>12})\tscore:\t{:0.3}'.format(word, p.normal_form, p.score))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/yermakhan/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [31]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [ ]:
import ssl
import certifi
from urllib.request import urlopen

request = "https://example.com"
urlopen(request, context=ssl.create_default_context

In [88]:
df

,dlg_id,line_n,role,text,is_greeting,is_introduction,manager_name,company_name,is_goodbye,lemmatized,lemmatized_string,bigram_list,trigram_list,fogram_list
0,0,0,manager,Алло,False,False,,,False,[алло],алло,[],[],[]
1,0,1,client,Алло здравствуйте,True,False,,,False,"[алло, здравствовать]",алло здравствовать,"[(алло, здравствовать)]",[],[]
2,0,2,manager,Добрый день,True,False,,,False,"[добрый, день]",добрый день,"[(добрый, день)]",[],[]
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,False,True,,,False,"[я, звать, ангелина, компания, диджитал, бизне...",я звать ангелина компания диджитал бизнес звон...,"[(я, звать), (звать, ангелина), (ангелина, ком...","[(я, звать, ангелина), (звать, ангелина, компа...","[(я, звать, ангелина, компания), (звать, ангел..."
4,0,4,manager,Ага,False,False,,,False,[ага],ага,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,client,По поводу виджетов и с ними уже обсудите конкр...,False,False,,,False,"[по, повод, виджет, и, с, они, уже, обсудить, ...",по повод виджет и с они уже обсудить конкретно...,"[(по, повод), (повод, виджет), (виджет, и), (и...","[(по, повод, виджет), (повод, виджет, и), (вид...","[(по, повод, виджет, и), (повод, виджет, и, с)..."
476,5,139,client,Все я вам высылаю счет и с вами на связи если ...,False,False,,,False,"[весь, я, вы, высылать, счет, и, с, вы, на, св...",весь я вы высылать счет и с вы на связь если б...,"[(весь, я), (я, вы), (вы, высылать), (высылать...","[(весь, я, вы), (я, вы, высылать), (вы, высыла...","[(весь, я, вы, высылать), (я, вы, высылать, сч..."
477,5,140,manager,Спасибо спасибо,False,False,,,False,"[спасибо, спасибо]",спасибо спасибо,"[(спасибо, спасибо)]",[],[]
478,5,141,manager,Да да тогда созвонимся ага спасибо вам давайте,False,False,,,False,"[да, да, тогда, созвониться, ага, спасибо, вы,...",да да тогда созвониться ага спасибо вы давать,"[(да, да), (да, тогда), (тогда, созвониться), ...","[(да, да, тогда), (да, тогда, созвониться), (т...","[(да, да, тогда, созвониться), (да, тогда, соз..."
